In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import cv2
from PIL import Image
from sklearn.metrics import accuracy_score

In [2]:
train_path = './Dataset_2/Train/'
test_path = 'Dataset_2/Test/'
data_dir = './Dataset_2'

train_lable = 'Dataset_2/Train.csv'
test_lable = 'Dataset_2/Test.csv'

In [3]:
NUM_CATEGORIES = len(os.listdir(train_path))
NAME_CATEGORIES = os.listdir(train_path)
print(NUM_CATEGORIES)
print(NAME_CATEGORIES)

43
['12', '16', '19', '32', '28', '30', '40', '24', '4', '2', '14', '38', '9', '20', '3', '41', '7', '0', '17', '39', '34', '22', '33', '8', '42', '1', '5', '27', '31', '26', '10', '25', '35', '6', '36', '15', '37', '23', '21', '18', '11', '29', '13']


In [4]:
# Resizing the images to 30x30x3
IMG_HEIGHT = 30
IMG_WIDTH = 30
channels = 3

In [5]:
# move resize
image_data = []
image_labels = []

for i in range(NUM_CATEGORIES):
    path = data_dir + '/Train/' + str(i)
    images = os.listdir(path)
    for img in images:
        try:
            image = cv2.imread(path + '/' + img)
            image_fromarray = Image.fromarray(image, 'RGB')
            #列表不够整齐，如果创建多维列表，子列表中的元素得个数一致，如果不一致，那么默认还是一维列表，把子列表当做元素而已
            resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))      
            imgray = cv2.cvtColor(np.array(resize_image), cv2.COLOR_BGR2GRAY)
            flat_arr=imgray.ravel()
            image_data.append(flat_arr)
            image_labels.append(i)
        except:
            print("Error in " + img)

In [6]:
test = pd.read_csv(data_dir + '/Test.csv')
labels = test["ClassId"].values
imgs = test["Path"].values
test_data =[]
for img in imgs:
    try:
        image = cv2.imread(data_dir + '/' +img)
        image_fromarray = Image.fromarray(image, 'RGB')
        resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
        imgray = cv2.cvtColor(np.array(resize_image), cv2.COLOR_BGR2GRAY)
        flat_arr=imgray.ravel()
        test_data.append(np.array(flat_arr))
    except:
        print("Error in " + img)

In [7]:
X_test = np.array(test_data)
X_test = X_test/255

In [8]:
img_dict={"images":np.array(imgray),"data":np.array(image_data),"target":np.array(image_labels)}
train = img_dict['data']/255

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, img_dict['target'], test_size=0.3, random_state=0, shuffle=True)

In [12]:
# 95% of variance
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95)
pca.fit(X_train)
reduced = pca.transform(X_train)
reduced

array([[-4.1755464 , -0.95518154, -0.01031487, ...,  0.23606274,
         0.03893254, -0.11832849],
       [-4.68560555,  0.5936293 ,  0.23502063, ..., -0.01153481,
         0.24389798,  0.02797636],
       [ 1.69043953, -3.29197593, -2.74476812, ..., -0.11696644,
        -0.00920289, -0.61913339],
       ...,
       [-2.28786161, -4.83753363,  1.36597034, ..., -0.12577415,
        -0.2078356 ,  0.35491813],
       [-0.06144099,  1.80547103,  2.37042   , ...,  0.09399711,
         0.16800563, -0.16032222],
       [-4.36964589, -3.57695685, -0.38318746, ...,  0.09788091,
        -0.10528912, -0.09641381]])

# 90-100

In [16]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
components_pca=pd.DataFrame(columns=["n_components","SVC_score","knn_score"])

for i in range(90,101):
    print("dealing with k = {}".format(i))
    pca=PCA(n_components=i,whiten=True,random_state=0).fit(X_train)
    X_train_pca=pca.transform(X_train)
    X_val_pca=pca.transform(X_val)

    clf=SVC(kernel="linear",random_state=0)
    clf.fit(X_train_pca, y_train)
    clf_predict=clf.predict(X_val_pca)
    
    knn=KNeighborsClassifier(n_neighbors=9)
    knn.fit(X_train_pca,y_train)
    knn_predict=knn.predict(X_val_pca)
    components_pca=components_pca.append([{"n_components":i+1,"SVC_score":accuracy_score(y_val,clf_predict),"knn_score":accuracy_score(y_val,knn_predict)}], ignore_index=True)

dealing with k = 90
dealing with k = 91
dealing with k = 92
dealing with k = 93
dealing with k = 94
dealing with k = 95
dealing with k = 96
dealing with k = 97
dealing with k = 98
dealing with k = 99
dealing with k = 100


In [17]:
components_pca

,n_components,SVC_score,knn_score
0,91,0.915328,0.812548
1,92,0.915838,0.814163
2,93,0.913202,0.812718
3,94,0.916093,0.809317
4,95,0.915753,0.814758
5,96,0.915413,0.814078
6,97,0.916348,0.813483
7,98,0.914478,0.813908
8,99,0.916858,0.814588
9,100,0.917283,0.813143


In [18]:
components_pca[components_pca['knn_score']==components_pca['knn_score'].max()]

,n_components,SVC_score,knn_score
4,95,0.915753,0.814758


In [19]:
components_pca[components_pca['SVC_score']==components_pca['SVC_score'].max()]

,n_components,SVC_score,knn_score
10,101,0.919918,0.813143


# 50-60

In [24]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
components_pca=pd.DataFrame(columns=["n_components","SVC_score","knn_score"])

for i in range(50,60):
    print("dealing with k = {}".format(i))
    pca=PCA(n_components=i,whiten=True,random_state=0).fit(X_train)
    X_train_pca=pca.transform(X_train)
    X_val_pca=pca.transform(X_val)

    clf=SVC(kernel="linear",random_state=0)
    clf.fit(X_train_pca, y_train)
    clf_predict=clf.predict(X_val_pca)
    
    knn=KNeighborsClassifier(n_neighbors=9)
    knn.fit(X_train_pca,y_train)
    knn_predict=knn.predict(X_val_pca)
    components_pca=components_pca.append([{"n_components":i+1,"SVC_score":accuracy_score(y_val,clf_predict),"knn_score":accuracy_score(y_val,knn_predict)}], ignore_index=True)

dealing with k = 50
dealing with k = 51
dealing with k = 52
dealing with k = 53
dealing with k = 54
dealing with k = 55
dealing with k = 56
dealing with k = 57
dealing with k = 58
dealing with k = 59


In [25]:
components_pca

,n_components,SVC_score,knn_score
0,51,0.871376,0.807107
1,52,0.873927,0.807192
2,53,0.874607,0.805577
3,54,0.882173,0.809827
4,55,0.883363,0.809062
5,56,0.883363,0.806767
6,57,0.884638,0.806342
7,58,0.887019,0.808297
8,59,0.889569,0.810423
9,60,0.891864,0.808127


# 100-120

In [29]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
components_pca=pd.DataFrame(columns=["n_components","SVC_score","knn_score"])

for i in range(100,120):
    print("dealing with k = {}".format(i))
    pca=PCA(n_components=i,whiten=True,random_state=0).fit(X_train)
    X_train_pca=pca.transform(X_train)
    X_val_pca=pca.transform(X_val)

    clf=SVC(kernel="linear",random_state=0)
    clf.fit(X_train_pca, y_train)
    clf_predict=clf.predict(X_val_pca)
    
    knn=KNeighborsClassifier(n_neighbors=9)
    knn.fit(X_train_pca,y_train)
    knn_predict=knn.predict(X_val_pca)
    components_pca=components_pca.append([{"n_components":i+1,"SVC_score":accuracy_score(y_val,clf_predict),"knn_score":accuracy_score(y_val,knn_predict)}], ignore_index=True)

dealing with k = 100
dealing with k = 101
dealing with k = 102
dealing with k = 103
dealing with k = 104
dealing with k = 105
dealing with k = 106
dealing with k = 107
dealing with k = 108
dealing with k = 109
dealing with k = 110
dealing with k = 111
dealing with k = 112
dealing with k = 113
dealing with k = 114
dealing with k = 115
dealing with k = 116
dealing with k = 117
dealing with k = 118
dealing with k = 119


In [30]:
components_pca

,n_components,SVC_score,knn_score
0,101,0.917283,0.813143
1,102,0.919918,0.813143
2,103,0.917538,0.811528
3,104,0.917198,0.811443
4,105,0.920513,0.815523
5,106,0.918983,0.813313
6,107,0.915753,0.813228
7,108,0.918898,0.817394
8,109,0.919408,0.812378
9,110,0.918133,0.812803


# 200-210

In [10]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
components_pca=pd.DataFrame(columns=["n_components","SVC_score","knn_score"])

for i in range(200,210):
    print("dealing with k = {}".format(i))
    pca=PCA(n_components=i,whiten=True,random_state=0).fit(X_train)
    X_train_pca=pca.transform(X_train)
    X_val_pca=pca.transform(X_val)

    clf=SVC(kernel="linear",random_state=0)
    clf.fit(X_train_pca, y_train)
    clf_predict=clf.predict(X_val_pca)
    
    knn=KNeighborsClassifier(n_neighbors=9)
    knn.fit(X_train_pca,y_train)
    knn_predict=knn.predict(X_val_pca)
    components_pca=components_pca.append([{"n_components":i+1,"SVC_score":accuracy_score(y_val,clf_predict),"knn_score":accuracy_score(y_val,knn_predict)}], ignore_index=True)

dealing with k = 200
dealing with k = 201
dealing with k = 202
dealing with k = 203
dealing with k = 204
dealing with k = 205
dealing with k = 206
dealing with k = 207
dealing with k = 208
dealing with k = 209


In [11]:
components_pca

,n_components,SVC_score,knn_score
0,201,0.905466,0.792825
1,202,0.909972,0.791635
2,203,0.909887,0.789594
3,204,0.910227,0.791040
4,205,0.909122,0.793080
5,206,0.910397,0.791465
6,207,0.909717,0.788744
7,208,0.907507,0.790020
8,209,0.907592,0.789935
9,210,0.912097,0.788999


# 证明：PCA 优化是凸优化吗？
https://stats.stackexchange.com/questions/301532/is-pca-optimization-convex

# final

In [22]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
components_pca=pd.DataFrame(columns=["n_components","SVC_score","knn_score","RandomForest_score","GaussianNB_score","Adaboost_score"])


In [23]:
components_pca

,n_components,SVC_score,knn_score,RandomForest_score,GaussianNB_score,Adaboost_score


# remain to deal with